In [9]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from keras.preprocessing import image
from tensorflow.keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
from PIL import Image

from PIL import Image, ImageOps
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

# Load the pre-trained VGG16 model
model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

def extract_features(img_path):
    img = Image.open(img_path)
    img = ImageOps.expand(img, border=(10, 10), fill='white') # Add 10 pixels of white padding around the image
    img = img.resize((224, 224)) # Resize the image to 224x224
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    return features

# Extract signatures from the two input images
signature1_path = '/content/drive/MyDrive/SML_mini/Class_pdf_jpg/1.jpg'
signature2_path = '/content/drive/MyDrive/SML_mini/input/1.jpg'
signature1 = extract_features(signature1_path)
signature2 = extract_features(signature2_path)

# Compare the two input signatures with all signatures in a folder
folder_path1 = '/content/drive/MyDrive/SML_mini/Class_pdf_jpg'
similarity_scores1 = []
for filename in os.listdir(folder_path1):
    if filename.endswith('.jpg'):
        signature_path1 = os.path.join(folder_path1, filename)
        signature = extract_features(signature_path1)
        similarity_score1 = np.dot(signature1.flatten(), signature.flatten()) / (np.linalg.norm(signature1) * np.linalg.norm(signature))
        similarity_scores1.append((signature_path1, similarity_score1))


# Compare the two input signatures with all signatures in a folder
folder_path2 = '/content/drive/MyDrive/SML_mini/input'
similarity_scores2 = []
for filename in os.listdir(folder_path2):
    if filename.endswith('.jpg'):
        signature_path2 = os.path.join(folder_path2, filename)
        signature = extract_features(signature_path2)
        similarity_score2 = np.dot(signature2.flatten(), signature.flatten()) / (np.linalg.norm(signature2) * np.linalg.norm(signature))
        similarity_scores2.append((signature_path2, similarity_score2))

# Find the signature image in the folder with the highest similarity score with the two input signature images
best_match1 = max(similarity_scores1, key=lambda x: x[1])
best_match2 = max(similarity_scores2, key=lambda x: x[1])

# Display the matched signature image and the similarity score
print('Best Match:', best_match1[0])
print('Similarity Score:', best_match1[1])

# Show the matched image
matched_image = Image.open(best_match1[0])
matched_image.show()